In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np

2025-10-07 14:17:21.497154: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-07 14:17:21.606723: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-07 14:17:21.682831: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759826841.770122   43578 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759826841.797824   43578 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1759826841.971268   43578 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [2]:
path_to_file = tf.keras.utils.get_file("shakespeare.txt", "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt")

text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

In [3]:
chars = sorted(set(text))
char2idx = {c: i for i,c in enumerate(chars)}
idx2char = np.array(chars)

In [4]:
seq_length = 100
sequences = []
for i in range(len(text) - seq_length):
    sequences.append([char2idx[c] for c in text[i:i+seq_length]])

In [5]:
X = np.array([seq[:-1] for seq in sequences])
y = np.array([seq[-1] for seq in sequences])
y = to_categorical(y,num_classes = len(chars))

In [6]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(len(chars), 64),
    tf.keras.layers.LSTM(256, return_sequences=True),
    tf.keras.layers.LSTM(256),
    tf.keras.layers.Dense(len(chars), activation='softmax')
])

2025-10-07 14:17:45.883890: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2025-10-07 14:17:45.883912: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:178] verbose logging is disabled. Rerun with verbose logging (usually --v=1 or --vmodule=cuda_diagnostics=1) to get more diagnostic output from this module
2025-10-07 14:17:45.883916: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:183] retrieving CUDA diagnostic information for host: kingstein
2025-10-07 14:17:45.883919: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:190] hostname: kingstein
2025-10-07 14:17:45.884017: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:197] libcuda reported version is: 575.64.3
2025-10-07 14:17:45.884031: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:201] kernel reported version is: 575.64.3
202

In [ ]:
model.compile(loss = 'categorical_crossentropy',optimizer = 'adam',metrics=['accuracy'])
model.fit(X,y,epochs = 1, batch_size = 128)

2025-10-07 14:17:47.059619: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 883312848 exceeds 10% of free system memory.


1581/8714 ━━━━━━━━━━━━━━━━━━━━ 51:33 434ms/step - accuracy: 0.2600 - loss: 2.7214

In [ ]:
def generate_text(seed_text,num_chars):
    for _ in range(num_chars):
        seed_seq = np.array([char2idx[c] for c in seed_text[-seq_length+1: ]])
        seed_seq = seed_seq.reshape(1,-1)
        pred = model.predict(seed_seq)
        next_char = idx2char[np.argmax(pred)]
        seed_text += next_char
    return seed_text

print(generate_text("To be, or not to be, that is the question:", 100) )